# SOTA ZDT Benchmarks

This notebook records the published wall-clock time (in hours) and evaluation counts achieved by the in-house NSGA-II implementation and the AutoNSGA-II configuration sweep across several large-dimensional ZDT problems.

| Problem | Variables | Time (h) NSGA-II | Time (h) AutoNSGA-II | Evaluations NSGA-II | Evaluations AutoNSGA-II |
|---------|-----------|------------------|----------------------|---------------------|-------------------------|
| ZDT1    | 2,048     | 0.13             | 0.02                 | 1,250,500           | 182,356                 |
| ZDT1    | 4,096     | 0.51             | 0.12                 | 2,906,100           | 484,356                 |
| ZDT1    | 8,192     | 2.40             | 0.50                 | 6,622,600           | 1,039,156               |
| ZDT1    | 16,384    | 11.19            | 2.15                 | 14,741,200          | 2,180,656               |
| ZDT1    | 32,768    | -                | 9.04                 | -                   | 4,605,556               |
| ZDT1    | 65,356    | -                | 31.66                | -                   | 9,494,556               |
| ZDT1    | 131,072   | -                | 120.02               | -                   | 19,359,356              |
| ZDT2    | 2,048     | 0.14             | 0.02                 | 1,472,800           | 164,756                 |
| ZDT2    | 4,096     | 0.62             | 0.10                 | 3,433,100           | 429,156                 |
| ZDT2    | 8,192     | 2.77             | 0.49                 | 7,676,600           | 986,556                 |
| ZDT2    | 16,384    | 12.30            | 2.28                 | 17,059,600          | 2,358,056               |
| ZDT2    | 32,768    | -                | 9.28                 | -                   | 4,736,056               |
| ZDT2    | 65,356    | -                | 39.19                | -                   | 10,081,856              |
| ZDT2    | 131,072   | -                | 138.85               | -                   | 21,703,556              |
| ZDT3    | 2,048     | 0.10             | 0.03                 | 1,089,800           | 253,356                 |
| ZDT3    | 4,096     | 0.47             | 0.16                 | 2,514,200           | 610,956                 |
| ZDT3    | 8,192     | 2.08             | 0.62                 | 5,463,000           | 1,267,656               |
| ZDT3    | 16,384    | 9.18             | 2.68                 | 11,877,500          | 2,820,556               |
| ZDT3    | 32,768    | -                | 11.39                | -                   | 6,158,256               |
| ZDT3    | 65,356    | -                | 40.69                | -                   | 11,912,856              |
| ZDT3    | 131,072   | -                | -                    | -                   | -                       |
| ZDT4*   | 2,048     | -                | 2.62                 | -                   | 21,746,882              |
| ZDT6    | 2,048     | 0.45             | 0.04                 | 5,401,100           | 291,856                 |
| ZDT6    | 4,096     | 1.82             | 0.16                 | 11,482,400          | 659,956                 |
| ZDT6    | 8,192     | 7.16             | 0.66                 | 24,897,300          | 1,374,056               |
| ZDT6    | 16,384    | -                | 3.08                 | -                   | 3,221,156               |
| ZDT6    | 32,768    | -                | 15.51                | -                   | 7,941,156               |
| ZDT6    | 65,356    | -                | 63.79                | -                   | 17,685,556              |
| ZDT6    | 131,072   | -                | -                    | -                   | -                       |

\* ZDT4 timing/evaluation data was only reported for AutoNSGA-II.


## Reproducing the NSGA-II baseline (full evaluation budget)

The table above is based on the published NSGA-II defaults (population 100, SBX crossover,
polynomial mutation, tournament size 2). The cell below launches the same configuration
for ZDT1 with 2,048 variables. It now uses the MooCore backend and stops automatically
when the hypervolume reaches 95% of the reference front (mirroring the paper's criterion).
**Expect this to take close to an hour on a CPU-only machine.**

Re-run the cell whenever you want to refresh the measurement; the notebook will read the
latest `metadata.json` emitted under `results/ZDT1/nsgaii/moocore/seed_*`.


In [1]:
import os
import subprocess
import sys
import time
from pathlib import Path

PROJECT_ROOT = Path.cwd()
env = os.environ.copy()
env.setdefault('PYTHONPATH', str(PROJECT_ROOT / 'src'))

cmd = [
    sys.executable, '-m', 'vamos.main',
    '--problem', 'zdt1',
    '--engine', 'moocore_v5',    
    '--n-var', '2048',
    '--max-evaluations', '12500500',
    '--hv-threshold', '0.95',
    '--population-size', '100',
    '--offspring-population-size', '100',
    '--nsgaii-crossover', 'sbx',
    '--nsgaii-crossover-prob', '0.9',
    '--nsgaii-crossover-eta', '20',
    '--nsgaii-mutation', 'pm',
    '--nsgaii-mutation-prob', '1/n',
    '--nsgaii-mutation-eta', '20',
    '--nsgaii-repair', 'random',
    '--selection-pressure', '2',
]
print('Running NSGA-II baseline with full evaluation budget...')
print(' '.join(cmd))
start = time.time()
subprocess.run(cmd, env=env, check=True)
elapsed = time.time() - start
print(f'\nCompleted in {elapsed / 3600:.2f} hours (~{elapsed / 60:.1f} minutes).')


Running NSGA-II baseline with full evaluation budget...
c:\Users\nicol\Desktop\VAMOS\.venv\Scripts\python.exe -m vamos.main --problem zdt1 --engine moocore_v5 --n-var 2048 --max-evaluations 12500500 --hv-threshold 0.95 --population-size 100 --offspring-population-size 100 --nsgaii-crossover sbx --nsgaii-crossover-prob 0.9 --nsgaii-crossover-eta 20 --nsgaii-mutation pm --nsgaii-mutation-prob 1/n --nsgaii-mutation-eta 20 --nsgaii-repair random --selection-pressure 2

Completed in 0.07 hours (~3.9 minutes).


In [2]:
from pathlib import Path
PROJECT_ROOT = Path(__file__).resolve().parents[1]

def find_latest_metadata():
    root = PROJECT_ROOT / 'results' / 'ZDT1' / 'nsgaii'
    if not root.exists():
        return None
    candidates = []
    for backend_dir in root.iterdir():
        if not backend_dir.is_dir():
            continue
        for seed_dir in backend_dir.glob('seed_*'):
            meta_path = seed_dir / 'metadata.json'
            if not meta_path.exists():
                continue
            with meta_path.open('r', encoding='utf-8') as fh:
                meta = json.load(fh)
            if (meta.get('problem', {}).get('key') == 'zdt1'
                    and meta.get('problem', {}).get('n_var') == 2048
                    and meta.get('max_evaluations') == 12500500):
                candidates.append((meta_path.stat().st_mtime, meta))
    return max(candidates, default=(None, None))[1]

meta = find_latest_metadata()
if meta:
    hours = meta['metrics']['time_ms'] / 1000 / 3600
    evals = meta['metrics']['evaluations']
    df.loc[(df['Problem'] == 'ZDT1') & (df['Variables'] == 2048), 'NSGA-II-Nico'] = f"{hours:.2f} h / {evals:,}"
else:
    print('No matching metadata found. Run the experiment cell above first.')

df


,Problem,Variables,Time (h) NSGA-II,Time (h) AutoNSGA-II,Evaluations NSGA-II,Evaluations AutoNSGA-II,NSGA-II-Nico
0,ZDT1,2048,0.13,0.02,1250500.0,182356.0,"0.06 h / 647,400"
1,ZDT1,4096,0.51,0.12,2906100.0,484356.0,
2,ZDT1,8192,2.40,0.50,6622600.0,1039156.0,
3,ZDT1,16384,11.19,2.15,14741200.0,2180656.0,
4,ZDT1,32768,NaN,9.04,NaN,4605556.0,
5,ZDT1,65356,NaN,31.66,NaN,9494556.0,
6,ZDT1,131072,NaN,120.02,NaN,19359356.0,
7,ZDT2,2048,0.14,0.02,1472800.0,164756.0,
8,ZDT2,4096,0.62,0.10,3433100.0,429156.0,
9,ZDT2,8192,2.77,0.49,7676600.0,986556.0,
